# Simple analysis of HTTP requests to the NASA Kennedy Space Center WWW server logs

### About

The objective of this project is to do some basic analysis of access logs from one of NASA's website.

This analysis should answer some proposed questions using Spark and Python3:
1. Number of unique hosts;
2. Total number of 404 errors;
3. Top 5 URLs with 404 error;
4. 404 errors occurrences by day;
5. Grand total of bytes returned;

### What was used

The used data can be downloaded from 

ftp://ita.ee.lbl.gov/traces/NASA_access_log_Jul95.gz

ftp://ita.ee.lbl.gov/traces/NASA_access_log_Aug95.gz

More information about the data can be found in this [link](http://ita.ee.lbl.gov/html/contrib/NASA-HTTP.html).

### Final Thoughts

The most probable source of errors in this kind of task would be the log parsing, because it is sort of text analysis and the regular expressions must be on point.

In [1]:
# IMPORTS
from pyspark.sql import SparkSession
import datetime
import pyspark.sql.functions as sparkF
import pyspark.sql.types as sparkT

In [2]:
# FUNCTIONS
DateParser = sparkF.udf(lambda x: datetime.datetime.strptime(x, '%d/%b/%Y:%H:%M:%S %z'), sparkT.DateType())

In [3]:
# INITIALIZATIONS
spark = SparkSession.builder.getOrCreate()

In [4]:
# PREPARING DATA
data_locations = [
                    "./NASA_access_log_Jul95.gz",
                    "./NASA_access_log_Aug95.gz"
                 ]
# iterating over data sources
logs = spark.read.text(data_locations[0])
if len(data_locations) > 1:
    for loc in data_locations[1:]:
        logs = logs.union(spark.read.text(loc))

In [5]:
# row number
logs.count()

3461613

In [6]:
# dataframe head
logs.show(6, truncate=90)

+------------------------------------------------------------------------------------------+
|                                                                                     value|
+------------------------------------------------------------------------------------------+
|    199.72.81.55 - - [01/Jul/1995:00:00:01 -0400] "GET /history/apollo/ HTTP/1.0" 200 6245|
|unicomp6.unicomp.net - - [01/Jul/1995:00:00:06 -0400] "GET /shuttle/countdown/ HTTP/1.0...|
|199.120.110.21 - - [01/Jul/1995:00:00:09 -0400] "GET /shuttle/missions/sts-73/mission-s...|
|burger.letters.com - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/countdown/liftoff.htm...|
|199.120.110.21 - - [01/Jul/1995:00:00:11 -0400] "GET /shuttle/missions/sts-73/sts-73-pa...|
|burger.letters.com - - [01/Jul/1995:00:00:12 -0400] "GET /images/NASA-logosmall.gif HTT...|
+------------------------------------------------------------------------------------------+
only showing top 6 rows



In [7]:
# parsing data
logs_data = logs\
            .select(
                sparkF.regexp_extract('value', '^([^\s]+)', 1).alias('host'),
                DateParser(sparkF.regexp_extract('value', '\[(.*?)\]', 1)).alias('timestamp'),
                sparkF.regexp_extract('value', '\"[^\s]+ ([^\s]+) [^\s]+\"', 1).alias('request_url'),
                sparkF.regexp_extract('value', '\"\s([^\s]+)', 1).alias('return_code'),
                sparkF.regexp_extract('value', '([^\s]+)$', 1).cast("integer").alias('size')
            )\
            .fillna(0, subset='size')
# DateParser function is slow but do the job
# error is thrown if this DF gets cached

In [8]:
# clean dataframe head
logs_data.show(6, truncate=45)

+--------------------+----------+---------------------------------------------+-----------+----+
|                host| timestamp|                                  request_url|return_code|size|
+--------------------+----------+---------------------------------------------+-----------+----+
|        199.72.81.55|1995-07-01|                             /history/apollo/|        200|6245|
|unicomp6.unicomp.net|1995-07-01|                          /shuttle/countdown/|        200|3985|
|      199.120.110.21|1995-07-01| /shuttle/missions/sts-73/mission-sts-73.html|        200|4085|
|  burger.letters.com|1995-07-01|              /shuttle/countdown/liftoff.html|        304|   0|
|      199.120.110.21|1995-07-01|/shuttle/missions/sts-73/sts-73-patch-smal...|        200|4179|
|  burger.letters.com|1995-07-01|                   /images/NASA-logosmall.gif|        304|   0|
+--------------------+----------+---------------------------------------------+-----------+----+
only showing top 6 rows



# Proposed Questions

### 1. Number of unique hosts

In [9]:
# logs_data.agg(sparkF.countDistinct(sparkF.col('host')).alias('distinct_hosts')).collect()  # another way of doing it
logs_data.select('host').distinct().count()
# 137979

137979

### 2. Total number of 404 errors

In [10]:
logs_data.filter(sparkF.col('return_code') == 404).count()
# 20873

20873

### 3. Top 5 URLs with 404 error

In [11]:
urls404 = logs_data\
            .filter(sparkF.col('return_code') == 404)\
            .groupby('request_url')\
            .count()\
            .sort(sparkF.col('count').desc())
# for row in urls404.collect()[:5]:  # printing only the urls
#     print(row['request_url'])
urls404.show(5, truncate=False)
# /pub/winvn/readme.txt
# /pub/winvn/release.txt
# /shuttle/missions/STS-69/mission-STS-69.html
# /shuttle/missions/sts-68/ksc-upclose.gif
# /history/apollo/a-001/a-001-patch-small.gif

+--------------------------------------------+-----+
|request_url                                 |count|
+--------------------------------------------+-----+
|/pub/winvn/readme.txt                       |2004 |
|/pub/winvn/release.txt                      |1732 |
|/shuttle/missions/STS-69/mission-STS-69.html|682  |
|/shuttle/missions/sts-68/ksc-upclose.gif    |426  |
|/history/apollo/a-001/a-001-patch-small.gif |384  |
+--------------------------------------------+-----+
only showing top 5 rows



### 4. 404 errors occurrences by day

In [12]:
urls404_by_day = logs_data\
                .filter(sparkF.col('return_code') == 404)\
                .groupby('timestamp')\
                .count()\
                .sort(sparkF.col('timestamp').asc())
urls404_by_day.show(20)

+----------+-----+
| timestamp|count|
+----------+-----+
|1995-07-01|  316|
|1995-07-02|  291|
|1995-07-03|  470|
|1995-07-04|  359|
|1995-07-05|  497|
|1995-07-06|  640|
|1995-07-07|  569|
|1995-07-08|  302|
|1995-07-09|  348|
|1995-07-10|  398|
|1995-07-11|  471|
|1995-07-12|  470|
|1995-07-13|  531|
|1995-07-14|  411|
|1995-07-15|  254|
|1995-07-16|  257|
|1995-07-17|  406|
|1995-07-18|  465|
|1995-07-19|  638|
|1995-07-20|  428|
+----------+-----+
only showing top 20 rows



### 5. Grand total of bytes returned

In [13]:
logs_data.groupBy().sum('size').collect()[0]['sum(size)']
# 65524314915

65524314915